<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_Temporal_Fusion_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
! pip install kaggle
! pip install wandb

y


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ekaterineghibradze","key":"b1414052fbae86987efff2083c8dcbd1"}'}

In [20]:
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [21]:
!ls -l ~/.kaggle/

total 4
-rw------- 1 root root 74 Jul  6 12:27 kaggle.json


In [22]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
! unzip walmart-recruiting-store-sales-forecasting

Archive:  walmart-recruiting-store-sales-forecasting.zip
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv.zip        
replace sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv.zip  
replace stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stores.csv              
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv.zip            
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv.zip           


In [24]:
!unzip '*.csv.zip'

y
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

4 archives were successfully processed.


In [25]:
!unzip '*.csv.zip'

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

4 archives were successfully processed.


In [26]:
import pandas as pd

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

# Explore the data
print("Train shape:", train.shape)
print(train.head())
print(train.columns)

print("\nTest shape:", test.shape)
print(test.head())
print(test.columns)

print("\nFeatures shape:", features.shape)
print(features.head())
print(features.columns)

print("\nStores shape:", stores.shape)
print(stores.head())
print(stores.columns)

print("\nSample Submission shape:", sample_submission.shape)
print(sample_submission.head())
print(sample_submission.columns)

Train shape: (421570, 5)
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')

Test shape: (115064, 4)
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
Index(['Store', 'Dept', 'Date', 'IsHoliday'], dtype='object')

Features shape: (8190, 12)
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        3

In [27]:
# Merge train and test with features and stores
train_merged = pd.merge(train, features, on=['Store', 'Date'], how='left')
train_merged = pd.merge(train_merged, stores, on='Store', how='left')

test_merged = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_merged = pd.merge(test_merged, stores, on='Store', how='left')

# Convert Date to datetime
train_merged['Date'] = pd.to_datetime(train_merged['Date'])


In [28]:
# Split
train_data = train_merged[train_merged['Date'] < '2012-01-01']
val_data = train_merged[(train_merged['Date'] >= '2012-01-01') & (train_merged['Date'] < '2012-07-01')]
test_data = train_merged[train_merged['Date'] >= '2012-07-01']

print("Train:", train_data.shape)
print("Validation:", val_data.shape)
print("Test (local):", test_data.shape)

Train: (294132, 17)
Validation: (77110, 17)
Test (local): (50328, 17)


In [29]:
import wandb
wandb.login()
wandb.init(project="Store-Sales-Forecasting", name="temporal-fusion-transformer-training-run")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eghib22 (eghib22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [30]:
def preprocess(df):
    df = df.copy()

    # Map Type A/B/C to 0/1/2
    type_map = {'A': 0, 'B': 1, 'C': 2}
    df['Type'] = df['Type'].map(type_map)

    # Ensure IsHoliday is int (True/False → 1/0)
    if 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].astype(int)
        df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].astype(int)

    # Fill NaNs in MarkDown columns with 0
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # Extract date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.day

    return df


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)


# WMAE calculation function
def wmae(y_true, y_pred, weights=None):
    """Weighted Mean Absolute Error"""
    if weights is None:
        weights = np.ones_like(y_true)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

class WalmartDataset(Dataset):
    """Custom dataset for Walmart sales data"""

    def __init__(self, data, sequence_length=8, prediction_length=1, target_col='Weekly_Sales'):
        self.data = data.copy()
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.target_col = target_col

        # Sort by Store, Dept, and time
        self.data = self.data.sort_values(['Store', 'Dept', 'Year', 'Month', 'Week'])

        # Create time index
        self.data['time_idx'] = self.data.groupby(['Store', 'Dept']).cumcount()

        # Determine feature columns once during initialization
        self._determine_feature_columns()

        # Get unique store-dept combinations
        self.store_dept_combinations = self.data[['Store', 'Dept']].drop_duplicates().reset_index(drop=True)

        # Create sequences
        self.sequences = []
        self.create_sequences()

    def _determine_feature_columns(self):
        """Determine which columns to use as features (done once during initialization)"""
        all_cols = list(self.data.columns)
        excluded_cols = [self.target_col, 'time_idx']

        # Remove excluded columns and any remaining non-numeric columns
        self.feature_cols = []
        skipped_cols = []

        for col in all_cols:
            if col not in excluded_cols:
                # Check if column contains numeric data
                try:
                    pd.to_numeric(self.data[col].iloc[0])
                    self.feature_cols.append(col)
                except (ValueError, TypeError):
                    skipped_cols.append(col)
                    continue

        # Print information only once
        if skipped_cols:
            print(f"Skipped non-numeric columns: {skipped_cols}")
        print(f"Using feature columns: {self.feature_cols}")
        print(f"Total features: {len(self.feature_cols)}")

    def create_sequences(self):
        """Create sequences for training"""
        for _, row in self.store_dept_combinations.iterrows():
            store, dept = row['Store'], row['Dept']

            # Get data for this store-dept combination
            store_dept_data = self.data[
                (self.data['Store'] == store) &
                (self.data['Dept'] == dept)
            ].reset_index(drop=True)

            # Create sequences
            for i in range(len(store_dept_data) - self.sequence_length - self.prediction_length + 1):
                seq_data = store_dept_data.iloc[i:i + self.sequence_length + self.prediction_length]

                if len(seq_data) == self.sequence_length + self.prediction_length:
                    self.sequences.append({
                        'store': store,
                        'dept': dept,
                        'sequence': seq_data
                    })

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence_data = self.sequences[idx]['sequence']

        # Split into input and target
        input_seq = sequence_data.iloc[:self.sequence_length]
        target_seq = sequence_data.iloc[self.sequence_length:self.sequence_length + self.prediction_length]

        # Use pre-determined feature columns
        input_data = input_seq[self.feature_cols].values.astype(np.float32)
        target_data = target_seq[self.target_col].values.astype(np.float32)

        # Convert to tensors
        X = torch.FloatTensor(input_data)
        y = torch.FloatTensor(target_data)

        # Static features (same for all time steps) - ensure they're numeric
        static_data = np.array([
            float(input_seq['Store'].iloc[0]),
            float(input_seq['Dept'].iloc[0]),
            float(input_seq['Type'].iloc[0]),
            float(input_seq['Size'].iloc[0])
        ], dtype=np.float32)

        static_features = torch.FloatTensor(static_data)

        return X, y, static_features

class MultiHeadAttention(nn.Module):
    """Multi-head attention mechanism"""

    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # Linear transformations
        Q = self.W_q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        # Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_k)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attention_weights = torch.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        context = torch.matmul(attention_weights, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Output projection
        output = self.W_o(context)

        # Residual connection and layer norm
        output = self.layer_norm(output + query)

        return output, attention_weights

class VariableSelectionNetwork(nn.Module):
    """Variable selection network for feature importance"""

    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        weights = torch.relu(self.linear1(x))
        weights = self.dropout(weights)
        weights = self.linear2(weights)
        weights = self.softmax(weights)

        # Apply weights to input
        selected_features = x * weights

        return selected_features, weights

class GatedResidualNetwork(nn.Module):
    """Gated Residual Network"""

    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.gate = nn.Linear(hidden_dim, output_dim)
        self.layer_norm = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout)

        # Skip connection
        if input_dim != output_dim:
            self.skip_connection = nn.Linear(input_dim, output_dim)
        else:
            self.skip_connection = None

    def forward(self, x):
        # Main path
        h = torch.relu(self.linear1(x))
        h = self.dropout(h)

        # Gate
        gate = torch.sigmoid(self.gate(h))

        # Output
        output = self.linear2(h) * gate

        # Skip connection
        if self.skip_connection is not None:
            residual = self.skip_connection(x)
        else:
            residual = x

        # Add residual and normalize
        output = self.layer_norm(output + residual)

        return output

class TemporalFusionTransformer(nn.Module):
    """Temporal Fusion Transformer for time series forecasting"""

    def __init__(self,
                 input_dim,
                 static_dim,
                 hidden_dim=128,
                 n_heads=8,
                 n_layers=3,
                 dropout=0.1,
                 prediction_length=1):
        super().__init__()

        self.input_dim = input_dim
        self.static_dim = static_dim
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.prediction_length = prediction_length

        # Variable selection networks
        self.variable_selection = VariableSelectionNetwork(
            input_dim, hidden_dim, input_dim, dropout
        )

        # Static feature processing
        self.static_encoder = GatedResidualNetwork(
            static_dim, hidden_dim, hidden_dim, dropout
        )

        # Temporal processing
        self.temporal_encoder = nn.LSTM(
            input_dim, hidden_dim, batch_first=True, dropout=dropout
        )

        # Multi-head attention layers
        self.attention_layers = nn.ModuleList([
            MultiHeadAttention(hidden_dim, n_heads, dropout)
            for _ in range(n_layers)
        ])

        # Position encoding
        self.position_encoding = nn.Parameter(torch.randn(1000, hidden_dim))

        # Output layers
        self.output_projection = nn.Sequential(
            GatedResidualNetwork(hidden_dim, hidden_dim, hidden_dim, dropout),
            nn.Linear(hidden_dim, prediction_length)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, static_features):
        batch_size, seq_len, _ = x.shape

        # Variable selection
        selected_features, selection_weights = self.variable_selection(x)

        # Static context
        static_context = self.static_encoder(static_features)

        # Temporal encoding
        temporal_features, (hidden, cell) = self.temporal_encoder(selected_features)

        # Add position encoding
        pos_encoding = self.position_encoding[:seq_len, :].unsqueeze(0).repeat(batch_size, 1, 1)
        temporal_features = temporal_features + pos_encoding

        # Multi-head attention
        attention_output = temporal_features
        for attention_layer in self.attention_layers:
            attention_output, _ = attention_layer(attention_output, attention_output, attention_output)

        # Combine with static context
        # Use the last time step for prediction
        final_features = attention_output[:, -1, :]  # (batch_size, hidden_dim)

        # Add static context
        static_context_expanded = static_context.unsqueeze(1).expand(-1, 1, -1)
        combined_features = final_features + static_context_expanded.squeeze(1)

        # Output projection
        output = self.output_projection(combined_features)

        return output, selection_weights

def train_tft_model(train_data, val_data, test_data, epochs=50, batch_size=32, learning_rate=0.001):
    """Train the TFT model"""

    print("Preprocessing data...")
    # Preprocess data
    train_processed = preprocess(train_data)
    val_processed = preprocess(val_data)
    test_processed = preprocess(test_data)

    print("Data shapes after preprocessing:")
    print(f"Train: {train_processed.shape}")
    print(f"Validation: {val_processed.shape}")
    print(f"Test: {test_processed.shape}")

    # Create datasets
    print("\nCreating datasets...")
    train_dataset = WalmartDataset(train_processed, sequence_length=8, prediction_length=1)
    val_dataset = WalmartDataset(val_processed, sequence_length=8, prediction_length=1)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Get feature dimensions
    sample_x, sample_y, sample_static = train_dataset[0]
    input_dim = sample_x.shape[1]
    static_dim = sample_static.shape[0]

    print(f"\nModel configuration:")
    print(f"Input dimension: {input_dim}")
    print(f"Static dimension: {static_dim}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

    # Initialize model
    model = TemporalFusionTransformer(
        input_dim=input_dim,
        static_dim=static_dim,
        hidden_dim=128,
        n_heads=8,
        n_layers=3,
        dropout=0.1,
        prediction_length=1
    )

    wandb.config={
          "learning_rate": 0.001,
          "epochs": 50,
          "batch_size": 32,
          "sequence_length": 8,
          "prediction_length": 1,
          "input_dim": input_dim,
          "static_dim": static_dim,
          "hidden_dim": 128,
          "n_heads": 8,
          "n_layers": 3,
          "dropout" :0.1,
      }
    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # Training loop
    train_losses = []
    val_losses = []

    best_val_loss = float('inf')
    best_model_state = None

    print(f"\nStarting training for {epochs} epochs...")
    print("-" * 60)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for batch_idx, (x, y, static) in enumerate(train_loader):
            optimizer.zero_grad()

            # Forward pass
            predictions, selection_weights = model(x, static)

            # Calculate loss
            loss = criterion(predictions.squeeze(), y.squeeze())

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for x, y, static in val_loader:
                predictions, _ = model(x, static)
                loss = criterion(predictions.squeeze(), y.squeeze())
                val_loss += loss.item()

                val_predictions.extend(predictions.squeeze().cpu().numpy())
                val_targets.extend(y.squeeze().cpu().numpy())

        # Calculate average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        wandb.log({
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "epoch": epoch + 1
        })

        # Calculate WMAE
        val_wmae = wmae(np.array(val_targets), np.array(val_predictions))
        wandb.log({"val_wmae": val_wmae})

        # Learning rate scheduling
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict().copy()

        # Print progress
        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f'Epoch [{epoch+1:3d}/{epochs}] | '
                  f'Train Loss: {avg_train_loss:.4f} | '
                  f'Val Loss: {avg_val_loss:.4f} | '
                  f'WMAE: {val_wmae:.4f} | '
                  f'LR: {optimizer.param_groups[0]["lr"]:.6f}')

    # After training, save the model
    wandb.save("best_model.pth")

    print("-" * 60)
    print("Training completed!")

    # Load best model
    model.load_state_dict(best_model_state)

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(train_losses, label='Train Loss')
    ax.plot(val_losses, label='Validation Loss')
    ax.set_title('Training History')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
    wandb.log({"loss_curve": wandb.Image(fig)})


    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.scatter(val_targets, val_predictions, alpha=0.5)
    plt.plot([min(val_targets), max(val_targets)], [min(val_targets), max(val_targets)], 'r--')
    plt.title('Validation: Actual vs Predicted')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return model, train_losses, val_losses

def generate_test_predictions(model, test_merged, sample_submission):
    """Generate predictions for test set"""

    # Preprocess test data
    test_processed = preprocess(test_merged)

    # Create dataset for prediction
    test_dataset = WalmartDataset(test_processed, sequence_length=8, prediction_length=1)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Generate predictions
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for x, _, static in test_loader:
            predictions, _ = model(x, static)
            all_predictions.extend(predictions.squeeze().cpu().numpy())

    # Create submission dataframe
    submission = sample_submission.copy()

    # Map predictions to submission format
    # This is a simplified mapping - you might need to adjust based on your data structure
    if len(all_predictions) <= len(submission):
        submission['Weekly_Sales'][:len(all_predictions)] = all_predictions
    else:
        submission['Weekly_Sales'] = all_predictions[:len(submission)]

    return submission

# Example usage:
if __name__ == "__main__":
    # Load your data (assuming you have the preprocessing function)
    # train_data, val_data, test_data should be your split data

    # Train the model
    print("Training Temporal Fusion Transformer...")
    model, train_losses, val_losses = train_tft_model(
        train_data, val_data, test_data,
        epochs=50,
        batch_size=32,
        learning_rate=0.001
    )

    # Generate test predictions
    print("Generating test predictions...")
    submission = generate_test_predictions(model, test_merged, sample_submission)

    # Save submission
    submission.to_csv('tft_submission.csv', index=False)
    print("Submission saved to 'tft_submission.csv'")

    # Calculate final WMAE on validation set
    val_dataset = WalmartDataset(preprocess(val_data), sequence_length=8, prediction_length=1)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model.eval()
    val_predictions = []
    val_targets = []

    with torch.no_grad():
        for x, y, static in val_loader:
            predictions, _ = model(x, static)
            val_predictions.extend(predictions.squeeze().cpu().numpy())
            val_targets.extend(y.squeeze().cpu().numpy())

    final_wmae = wmae(np.array(val_targets), np.array(val_predictions))
    print(f"Final Validation WMAE: {final_wmae:.4f}")

Training Temporal Fusion Transformer...
Preprocessing data...
Data shapes after preprocessing:
Train: (294132, 20)
Validation: (77110, 20)
Test: (50328, 20)

Creating datasets...
Skipped non-numeric columns: ['Date']
Using feature columns: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day']
Total features: 18
Skipped non-numeric columns: ['Date']
Using feature columns: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day']
Total features: 18

Model configuration:
Input dimension: 18
Static dimension: 4
Training samples: 268488
Validation samples: 52469

Starting training for 50 epochs...
------------------------------------------------------------
